# Retrieve numeric features representing the postures.

In this notebook I convert the images to numeric representations, using posture detection from mediapipe. To convert the retrieved landmarks to features, I have used a method described by Siam et al.(2022)

### Retrieving the landmarks and edges for all images
The function "draw_landmarks_on_image" takes an images and detection results as input and returns an annotaded image and save one to disk. Furthermore the model will be initialized. This code is copied from the MediaPipe example code: 
https://github.com/googlesamples/mediapipe/blob/main/examples/pose_landmarker/python/%5BMediaPipe_Python_Tasks%5D_Pose_Landmarker.ipynb


In [1]:
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import os
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

def draw_landmarks_on_image(rgb_image, output_dir, image_name, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
    
  # To Save the image with annotations, first ensure the 'processed' directory exists
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  output_path = os.path.join(output_dir, image_name)
  cv2.imwrite(output_path, annotated_image)

#Create a PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='C:/Users/bcrui/Documents/OU-ML/pose_landmarker_full.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

Much shorter than the above code, but this does same as the function above, but for the YOLO code.

In [2]:
from ultralytics import YOLO
from PIL import Image

def draw_keypoints_on_image(rgb_image, output_dir, detection_result):
    
    # To Save the image with annotations, first ensure the 'processed' directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # View results
    for r in detection_result:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image        
        output_path = os.path.join(output_dir, rgb_image)
        im.save(output_path)  # save image

### Converting the landmarks and Keypoints to features

In this part I converted the retrieved landmarks and keypoints to features, according to the work of Siam et al.(2022)

#### Point and vertices
I used almost all landmarks, focusing specifically on points related to the torso, arms, and legs, while excluding facial landmarks, as the images do not feature the actors' faces. The left image shows the landmarks from Mediapipe and the image left the keypoints from Yolov8
<table><tr>
<td> <img src="https://developers.google.com/static/mediapipe/images/solutions/pose_landmarks_index.png" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="YoloKeypoints.png" alt="Drawing" style="width: 250px;"/></br>
source:https://learnopencv.com/wp-content/uploads/2021/05/fix-overlay-issue.jpg </td></tr>
</table>

The function "calculate_angle_difference" below, calculates the points and edges to angles of 2 connected edges, that can be saved as features for classification.

The function "Retrieve_the_angles", calculates the angels of all 3 point combinations and will return a list with angles which can be used as features for classification.

In [3]:
import math

def calculate_angle_difference(x1, y1, x2, y2, x3, y3):
    
    #Calculate the difference between two angles formed by three points.

    # Calculate each angle using the arctangent function
    angle1 = math.atan2(y3 - y2, x3 - x2)
    angle2 = math.atan2(y1 - y2, x1 - x2)

    # Calculate the difference
    angle_difference = angle1 - angle2

    return angle_difference

def Retrieve_the_angles(data, model):
    
    if model=='mediapipe':
        #List of point combinations for mediapipe
        points_sets =[(18,16,20),(22,16,14),(16,14,12),(17,15,19),(21,15,13),(15,13,11),(14,12,24),(13,11,23),(24,12,11),
                    (12,11,23),(24,23,11),(23,24,12),(23,24,26),(25,23,24),(24,26,28),(23,25,27),(26,28,32),(30,32,28),(25,27,31),(29,31,27)]
    else:
         #List of point combinations for yolo
        points_sets =[(10,9,6),(8,6,12),(12,6,5),(6,5,11),(11,5,7),(5,7,9),(6,12,11),(5,11,12),(14,12,11),(13,11,12),(12,14,16),(11,13,15)]
    
    #calculate for all point combinations the angle
    angles=[]
    for points in points_sets:
        
        #retrieve the points from the angle_list
        p1 = points[0]
        p2 = points[1]
        p3 = points[2]

        #calculate the angle difference and append it to the list angles
        angle = calculate_angle_difference(data['x'][p1], data['y'][p1], data['x'][p2], data['y'][p2], data['x'][p3], data['y'][p3])
        
        # Check if the angle is less than 0, if so add 2*pi
        if angle < 0:
            angle += 2 * math.pi
        angles.append(angle)  
    
    #return the angels
    return angles
    

The 'Retrieve_feature' function retrieves the landmarks of an image and returns the features (the angles)

In [4]:
import os
import torch

def Retrieve_features(rgb_image,images_dir,output_dir,model):
    
    #Load the input image.
    image_path = os.path.join(images_dir, rgb_image)
    image = mp.Image.create_from_file(image_path)
    
    # Create a dictionary to hold the x and y coordinates
    data = {"x": [], "y": []}
    probs_media=[]
    probs_yolo=[]
    
    #Select the correct model
    if model=="mediapipe":
        
        #Detect pose landmarks from the input image.
        detection_result = detector.detect(image)

        # draw landmarks on image
        annotated_image = draw_landmarks_on_image(image.numpy_view(), output_dir, rgb_image, detection_result)
        
        # Retrieve the list of landmarks
        pose_landmarks = detection_result.pose_landmarks
       
        #if the landmarks are not null, retrieve the angles
        if pose_landmarks==[]:
            angles=[]
        else: 
            # Flatten the list of lists if needed
            if isinstance(pose_landmarks[0], list):
                pose_landmarks = [item for sublist in pose_landmarks for item in sublist]

            # Iterate over the landmarks and extract data
            for landmark in pose_landmarks:
                data["x"].append(landmark.x)
                data["y"].append(landmark.y)
                probs_media.append(float(landmark.visibility))

            #retrieve the angles if the landmarks are above a certain treshold
            if np.mean(probs_media)<0.80 and probs_media!=[]:
                angles=[]
            else:
                angles = Retrieve_the_angles(data,'mediapipe') 
    else:
        # Load a model
        model = YOLO('yolov8n-pose.pt')

        # Predict with the model
        results = model(image_path,verbose=False) 
        
        #create images with keypoints
        draw_keypoints_on_image(rgb_image, output_dir, results)
        
        for r in results:
            # Move the tensor to the CPU and convert it to a NumPy array, extract x,y and conf (probabilities of points)
            coordinates = r.keypoints.xy.cpu().numpy()
            probs_yolo.append(r.keypoints.conf.cpu().numpy())

            # Extract x and y coordinates and probabilities for each point
            x_coordinates = coordinates[0, :, 0]
            y_coordinates = coordinates[0, :, 1]

            # Create a dictionary with 'x' and 'y' keys
            data = {'x': x_coordinates.tolist(), 'y': y_coordinates.tolist()}

        #if the keypoints are not null and the probability of alle points nog below 0.8 retrieve the angles
        if data==[] or (np.mean(probs_yolo)<0.80 and probs_yolo!=[]):
            print(rgb_image)
            angles=[]
        else:
            angles = Retrieve_the_angles(data,'yolo')
    
    return angles
                


### Import the BESST dataset and correct mistakes in the dataset
First check for mistakes in the dataset of images that are absent in the dataset

In [5]:
import pandas as pd
from pathlib import Path

# Read the Excel file sheets
postureData = pd.ExcelFile('datasets/BESST_Data.xlsx')
posture_sheet1 = pd.read_excel(postureData, sheet_name= 'Frontal Bodies')
posture_sheet1['Facing']='Frontal'
posture_sheet2 = pd.read_excel(postureData, sheet_name= 'Averted Bodies')   
posture_sheet2['Facing']='Averted'

# Concatenate both sheets in a single dataframe
posture_df = pd.concat([posture_sheet1,posture_sheet2], ignore_index=True)

# set the directory with the images and for the processed images
notebook_dir = os.getcwd()
images_dir = Path(notebook_dir) / 'postures'

# Get a list of all files in the directory
all_files = os.listdir(images_dir )

# Find images that are in the variable but not in the directory
images_only_in_variable = set(posture_df['Image']) - set(all_files)

# Find images that are in the variable but not in the directory
images_only_in_directory = set(all_files) - set(posture_df['Image'])

print("Images only in directory:", images_only_in_directory)
print("Images only in variable:", images_only_in_variable)


Images only in directory: {'009wN_45.jpg', '074mN_45.jpg', '076mW_90.jpg', '024wA_45.jpg'}
Images only in variable: {'074mN_45.jp', '081mT_45.jpg', '024wA_45.jp', '009wN_45.jp', '076mW_90.jp'}


Correct wrong filenames in the dataset and remove all the rest

In [6]:
# Replace '.jp' with '.jpg' in the 'Image' column
posture_df['Image'] = posture_df['Image'].str.replace(r'\.jp$', '.jpg', regex=True)

# Remove rows where 'Image' is '081mT_45.jpg'
posture_df = posture_df[posture_df['Image'] != '081mT_45.jpg']

images_only_in_directory = set(all_files) - set(posture_df['Image'])

# Iterate over the list of files which are not in the dataset
for file_name in images_only_in_directory:
    file_path = os.path.join(images_dir, file_name)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed file: {file_path}")
    else:
        print(f"File not found: {file_path}")

# Check one's more
all_files = os.listdir(images_dir)
images_only_in_variable = set(posture_df['Image']) - set(all_files)
images_only_in_directory = set(all_files) - set(posture_df['Image'])

print("Images only in directory:", images_only_in_directory)
print("Images only in variable:", images_only_in_variable)

Images only in directory: set()
Images only in variable: set()


### Retrieving features for all images and adding to the dataset
Add the features for each observation, to the dataset.

On the printed pictures below (after running the code), no landmarks where found, so these were deleted from the dataset.

In [7]:
# set the directory with the images and for the processed images
notebook_dir = os.getcwd()
images_dir = Path(notebook_dir) / 'postures'
output_media = Path(notebook_dir) / 'processed_media'
output_yolo = Path(notebook_dir) / 'processed_yolo'

# Use lambda to pass the column value and the fixed variables
features_media = posture_df['Image'].apply(lambda x: Retrieve_features(x,images_dir,output_media,'mediapipe')).apply(pd.Series)
features_yolo = posture_df['Image'].apply(lambda x: Retrieve_features(x,images_dir,output_yolo,'yolo')).apply(pd.Series)

# Concatenate the results with the original DataFrame
result_media = pd.concat([posture_df, features_media], axis=1)
result_yolo = pd.concat([posture_df, features_yolo], axis=1)

# Identify the rows where no landmarks were detected
result_media = result_media.dropna()
result_yolo = result_yolo.dropna()

#save the new dataset to a csv file
result_media.to_csv('datasets/postures_media.csv' ,index=False) 
result_yolo.to_csv('datasets/postures_yolo.csv' ,index=False) 

C:\Users\bcrui\AppData\Local\Temp\ipykernel_31100\76570754.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  features_media = posture_df['Image'].apply(lambda x: Retrieve_features(x,images_dir,output_media,'mediapipe')).apply(pd.Series)


024wN_90.jpg
057mA_90.jpg
064mA_90.jpg
006wA_90.jpg
009wA_90.jpg
021mE_90.jpg
002wE_90.jpg
008wF_90.jpg
024wF_90.jpg
038wF_90.jpg
046wF_90.jpg
047wF_90.jpg
043mT_90.jpg
005wT_90.jpg
007wT_90.jpg
065mU_90.jpg
002wW_90.jpg
006wN_45.jpg
064mA_45.jpg
016wA_45.jpg
035wA_45.jpg
047wA_45.jpg
048wA_45.jpg
021mE_45.jpg
002wE_45.jpg
053mF_45.jpg
008wF_45.jpg
009wF_45.jpg
015wF_45.jpg
041wF_45.jpg
021mT_45.jpg
054mT_45.jpg
062mT_45.jpg
075mT_45.jpg
005wT_45.jpg
022wT_45.jpg
048wT_45.jpg
049wT_45.jpg
065mU_45.jpg
071mU_45.jpg


C:\Users\bcrui\AppData\Local\Temp\ipykernel_31100\76570754.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  features_yolo = posture_df['Image'].apply(lambda x: Retrieve_features(x,images_dir,output_yolo,'yolo')).apply(pd.Series)


In [ ]:
print(result_media.shape[0])
print(result_yolo.shape[0])

1069
1088
